In [ ]:
!module unload Python-Neuroimaging

In [ ]:
!pip freeze | grep mne

In [ ]:
import mne
assert mne.__version__ == '0.23.0'

In [ ]:
%load_ext autoreload
%autoreload 2

from utils_postprocess import printDict

import globvars as gv
import mne
import matplotlib.pyplot as plt
from os.path import join as pjoin


defdgen = lambda sz :np.random.uniform(low=-1,high=1, size=sz)

import utils_tSNE as utsne

sfreq = 256
sfreq_hires = 1024

from IPython import get_ipython; ipython = get_ipython()
ipython.magic('run -i ../test_data/gen_test_data4.py')

print(test_data_descr)

#rawnames = ['S01_off_move','S01_off_hold','S01_on_move']

# prep_test_data

# _gt means ground truth here
dat_pri_gt           = dat_pri
dat_LFP_hires_pri_gt = dat_LFP_hires_pri

plt.plot(times,dat_pri[dati].T)
plt.figure()
plt.plot(times_hires,dat_LFP_hires_pri[dati].T, label=sfo_LFP)
plt.legend()

dat_EMG = defdgen( (2, nbins ) )

# reversed if needed
from utils import makeSimpleRaw
from featlist import selFeatsRegexInds
raws_permod_both_sides2 = {}
for rawi,rawn in enumerate(rawnames):
    raws_permod_both_sides2[rawn] = {}
    chis_LFP = utsne.selFeatsRegexInds(sfo_pri[rawi], 'LFP.*' )
    dat_LFP = dat_pri_gt[rawi][chis_LFP]
    raws_permod_both_sides2[rawn]['LFP'] = makeSimpleRaw(dat_LFP, np.array(sfo_pri[rawi])[chis_LFP], rescale=0 )

    chis_msrc = utsne.selFeatsRegexInds(sfo_pri[rawi], 'msrc.*' )
    dat_msrc = dat_pri_gt[rawi][chis_msrc]
    raws_permod_both_sides2[rawn]['src'] = makeSimpleRaw(dat_msrc, 
        np.array(sfo_pri[rawi])[chis_msrc] , rescale=0)

    raws_permod_both_sides2[rawn]['LFP_hires'] = makeSimpleRaw(dat_LFP_hires_pri_gt[rawi],
        sfo_lfp_hires_pri[rawi], sfreq=sfreq_hires, rescale=0 )
    
    raws_permod_both_sides2[rawn]['EMG'] = makeSimpleRaw(dat_EMG, 
        gv.EMG_per_hand[main_side], sfreq=sfreq, rescale=0)

rawnames_str = ','.join(rawnames)

####################################
import globvars as gv
gv.DEBUG_MODE=1
from IPython import get_ipython; ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug
raws_permod_both_sides = raws_permod_both_sides2

mstr = 'run -i ../run/run_prep_dat.py -r ' +   rawnames_str + \
        ' --param_file prep_dat_defparams.ini --calc_stats_multi_band 1' +\
        ' --save_dat 1 --save_stats 1 --use_preloaded_raws 1 --allow_CUDA 0 --n_jobs 20'
        
ipython.magic(mstr)

# %run -i ../run/run_prep_dat.py -r S99_off_move,S99_off_hold,S99_on_move \
#         --param_file prep_dat_defparams.ini --calc_stats_multi_band 1 --save_dat 1 \
#         --use_preloaded_raws 1 --save_stats 1

In [ ]:
from IPython import get_ipython; ipython = get_ipython()
#debug

# import warnings
# warnings.filterwarnings("error")
# # with warnings.catch_warnings():
# #     warnings.simplefilter('error')

# try:

# except ComplexWarning as e:
#     pass

rs = ('%run -i ../run/run_genfeats.py '
' --sources_type parcel_aal --bands crude'
' --src_grouping_fn 10'
' --src_grouping 0 '
' --raw {} '
' --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr'
' --Kalman_smooth 0'
' --load_TFR 0'
' --load_CSD 0'
' --save_TFR 0'
' --save_CSD 0'
' --save_bpcorr 0'
' --save_rbcorr 0'
' --load_rbcorr 0'
' --use_existing_TFR 0'
' --use_preloaded_data 0'
' --allow_CUDA 0'
' --load_only 0'
' --show_plots 0'
' --plot_types ,'
' --prescale_data 1'
' --exit_after {}'
' --normalize_TFR separate'
' --scale_data_combine_type no'
' --baseline_int_type entire'
' --n_jobs 1'
' --save_feat 1'
' --feat_stats_artif_handling reject'
' --scale_data_combine_type no '
' --rbcorr_use_local_means 0'
' --output_subdir test'
' --stats_fn_prefix stats_{}_{}_ '
' --param_file genfeats_defparams.ini ')

#exit_after = 'TFR_and_CSD'
exit_after = 'end'
#exit_after = 'prescale_data'
#exit_after = 'load'
mstr2 = rs.format(rawnames_str,exit_after,rawnames_str[:3],len(rawnames))

#%debug

ipython.magic(mstr2)

#np.savez(pjoin(gv.data_dir,'test.npz') ,globals() )

In [ ]:
# search LFP
import globvars as gv
rawnames_str = 'S95_off_move'
gv.DEBUG_MODE=1

%load_ext autoreload
%autoreload 2

#%debug

from IPython import get_ipython; ipython = get_ipython()
mstr_ML = '''%run -i ../run/run_ML.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
    --src_grouping 0 \
    --prefix test \
    --skip_XGB 0\
    --mods LFP,msrc\
    --LFP_related_only 1\
    --feat_types rbcorr,bpcorr,con,Hjorth\
    --parcel_group_names motor-related\
    --allow_CUDA 0\
    --subskip_fit 1 \
    --n_splits 2 \
    --n_jobs 1\
    --calc_MI 0 \
    --save_output 1\
    --baseline_int_type entire\
    --heavy_fit_red_featset 0\ 
    --groupings_to_use merge_movements --int_types_to_use basic\
    --input_subdir test --output_subdir test_searchLFP\
    --raw {} --load_only 0 --show_plots 1\
    --param_file ML_defparams.ini \
    --featsel_method XGB_Shapley\
    --calc_VIF 0 --skip_LDA 1 --calc_Boruta 0\
    --exit_after {}\
    --prep_for_clf_only 0
    --search_best_LFP XGB'''

exit_after = 'end'
mstr_ML = mstr_ML.format(rawnames_str,exit_after)


#%debug
ipython.magic(mstr_ML)

In [ ]:
from utils_postprocess_HPC import collectBestLFP
%load_ext autoreload
%autoreload 2
#%debug
r = collectBestLFP('test_searchLFP', save_result = 1);

In [ ]:
rawnames_str = 'S95_off_move'
import globvars as gv
gv.DEBUG_MODE=1

%load_ext autoreload
%autoreload 2

from IPython import get_ipython; ipython = get_ipython()
mstr_ML = '''%run -i ../run/run_ML.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
    --src_grouping 0 \
    --prefix test \
    --skip_XGB 0\
    --mods LFP,msrc\
    --LFP_related_only 1\
    --feat_types rbcorr,bpcorr,con,Hjorth\
    --parcel_group_names motor-related\
    --allow_CUDA 0\
    --subskip_fit 1 \
    --n_splits 2 \
    --calc_MI 0 \
    --save_output 0\
    --baseline_int_type entire\
    --heavy_fit_red_featset 0\ 
    --groupings_to_use merge_movements --int_types_to_use basic\
    --input_subdir test --output_subdir test\
    --raw {} --load_only 0 --show_plots 1\
    --param_file ML_defparams.ini \
    --featsel_method XGB_Shapley\
    --LFPchan=main
    --best_LFP_info_file test_searchLFP/best_LFP_info_both_sides_ext.json
    --exit_after {}\
    --prep_for_clf_only 0'''

exit_after = 'end'
mstr_ML = mstr_ML.format(rawnames_str,exit_after)


#%debug
ipython.magic(mstr_ML)

In [ ]:
featinds_good_boruta

In [ ]:
featinds_ranking_boruta

In [ ]:
colinds_good_VFsel[featinds_good_boruta]


In [ ]:
Xconcat_good_cur.shape

In [ ]:
colinds_good_VFsel

In [ ]:
perfs_XGB[0].keys()

In [ ]:
perfs_XGB[-1].keys()

In [ ]:
featnames_nice

In [ ]:
Xconcat_good_cur

In [ ]:
colinds_bad_VFsel

In [ ]:
anndict_per_intcat_per_rawn

In [ ]:
np.nan

In [ ]:
discard_ratio

In [ ]:
mstr_ML

In [ ]:
discard_outliers_q

In [ ]:
artif_naive_bininds